In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import partitura as pt
import partitura.musicanalysis as musan
from ic.data import DataPiece, Piece
import importlib
from CIA.getters import get_dataloader_generator
from CIA.data_processors.data_processor import DataProcessor
from CIA.getters import get_data_processor
from pathlib import Path
from pretty_midi import PrettyMIDI
import json
import re
from ic.beam_search.io import ICRes
import torch
import os
import pretty_midi
from CIA.dataloaders.piano_dataloader import PianoDataloaderGenerator
from scipy.stats import pearsonr


In [5]:
batik_dir = '/share/hel/home/mathias/datasets/batik_plays_mozart'
midi_file = batik_dir+'/midi/kv331_1.mid'
perf1 = pt.load_performance_midi(midi_file)
match_file = batik_dir + '/match/kv331_1.match'
perf2, align = pt.load_match(match_file)

note_array = perf2.note_array()
n_notes = note_array.shape[0]
hehe_file = pretty_midi.PrettyMIDI(midi_file)
pitches = [n.pitch for n in sorted(hehe_file.instruments[0].notes, key=lambda x: x.start)]
# tt = musan.estimate_tonaltension(perf2)

/share/hel/home/mathias/miniconda3/envs/pia2/lib/python3.8/site-packages/partitura/io/importmidi.py:186: UserWarning: ignoring MIDI message note_off channel=1 note=64 velocity=0 time=64
  warnings.warn(f"ignoring MIDI message {msg}")


In [4]:
min_notes_in_context = 256
n_inpaint = 506
hop_size = n_inpaint-min_notes_in_context
hop_size, n_notes//hop_size
decoding_start = 514



In [8]:
n_notes//(n_inpaint-min_notes_in_context)

24

In [82]:
files = sorted(Path(batik_dir, 'midi').glob('*.mid'))
pieces = []
for f in files:
    n_notes = len(PrettyMIDI(str(f)).instruments[0].notes)
    for i in range(-min_notes_in_context, n_notes-n_inpaint, hop_size):
        pieces.append(dict(
            path=str(f),
            start_node=i,
            n_inpaint=n_inpaint,
            end_window=0,
        # ) for i in range(0-min_notes_in_context-1, n_notes, hop_size)],
        ))
pieces

with open('configs/batik_all_pieces.json', 'w') as f:
    json.dump(pieces, f, indent=2)

In [37]:
dp = DataPiece(
    label='batik',
    pieces=[Piece(
        midi_file,
        start_node=i,
        n_inpaint=n_inpaint,
        end_window=0,
    # ) for i in range(0-min_notes_in_context-1, n_notes, hop_size)],
    ) for i in range(-min_notes_in_context, n_notes, hop_size)],
    cache_path='/share/hel/home/mathias/.cache/mutdata/pia',
    # pad_before=True,
)
model_dir = 'models/piano_event_performer_2021-10-01_16:03:06'
config = importlib.import_module('.config_autoreg', f'{model_dir.replace("/", ".")}').config
for t in ['time_dilation', 'velocity_shift', 'transposition']:
    config['dataloader_generator_kwargs']['transformations'][t] = False
# NOTE: unconmment, since we indeed want to do only inpainting
# config['data_processor_kwargs']['num_events_before'] = 1
# config['data_processor_kwargs']['num_events_after'] = 0
# TODO: check correct number...
# num_events_middle = 1024 - 6
# num_events_middle = 512 - 5 # magic - 5 for special tokens
# num_events_middle = 256
num_max_generated_events = 1024 - 6

load = True
overfitted = True

# === Decoder ====
# dataloader generator
dg : PianoDataloaderGenerator =  get_dataloader_generator(
    dataset=config["dataset"],
    dataloader_generator_kwargs=config["dataloader_generator_kwargs"],
)
dpr : DataProcessor = get_data_processor(
        dataloader_generator=dg,
        data_processor_type=config["data_processor_type"],
        data_processor_kwargs=config["data_processor_kwargs"],
    )
dp.dataloader_generator = dg

Loading index dictionnary


In [123]:
windows = list(iter(dp))
torch.cuda.is_available = lambda : False
os.environ["CUDA_VISIBLE_DEVICES"] = ''
x, metadata_dict = zip(*[dpr.preprocess(window[0], num_events_middle=n_inpaint) for window in windows])


Loading index dictionnary


/share/hel/home/mathias/devel/python3/PIA.No3/ic/data.py:167: UserWarning: hard coded num_before tokens
  warn('hard coded num_before tokens')
/share/hel/home/mathias/devel/python3/PIA.No3/ic/data.py:176: UserWarning: This put's the first token of 
  warn('This put\'s the first token of ')
/share/hel/home/mathias/devel/python3/PIA.No3/ic/data.py:171: UserWarning: Test that the following line works by also when piece.start <=0
  warn('Test that the following line works by also when piece.start <=0')


In [119]:
x0 = x[0][0,decoding_start:decoding_start+n_inpaint,0]
x1 = x[1][0,decoding_start+n_inpaint-hop_size:decoding_start+n_inpaint,0]
x2 = x[2][0,decoding_start+n_inpaint-hop_size:decoding_start+n_inpaint,0]

In [126]:
cated = [x[0][decoding_start:decoding_start+n_inpaint]] + [x_[decoding_start+n_inpaint-hop_size:decoding_start+n_inpaint,0] for x_ in x[1:]]
res = torch.cat(cated)

In [127]:
(res[:len(pitches)] == torch.tensor([dg.dataset.value2index['pitch'][i] for i in pitches])).all()

tensor(True)

# Stich the ics together

In [88]:
out_dir = Path('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0')
pat = re.compile(r'^(?P<piece>.+)_start_(?P<start>-?\d+)_nodes_506_end_0$')
def parse_path(p : Path) -> int:
    match = pat.match(p.name)
    return match.group('piece'), int(match.group('start'))
paths = list(out_dir.glob('*_nodes_506_end_0'))
piece_dict = {}
for p, (piece, start) in zip(paths, [parse_path(p) for p in  paths]):
    piece_dict.setdefault(piece, []).append((p,start))
for key, val in piece_dict.items():
    val = sorted(val, key=lambda x: x[1])
    paths,_= zip(*val)
    piece_dict[key] = {'paths': paths}
for piece_name in sorted(piece_dict.keys()):
    toks = []
    tok_times = []
    tok_ics = []
    # [x[0][decoding_start:decoding_start+n_inpaint]] + [x_[decoding_start+n_inpaint-hop_size:decoding_start+n_inpaint,0] for x_ in x[1:]]
    piece = piece_dict[piece_name]
    for i, p in enumerate(piece['paths']):
        res_path = p.joinpath('temp', 'ic.pt')
        if res_path.exists():
            res = ICRes.load(str(res_path))
            if i == 0:
                ic_toks = res.ic_tok[:n_inpaint]
                real_n_inpaint = res.ic_tok.shape[0]
                toks.append(res.tok[decoding_start:decoding_start+real_n_inpaint])
                tok_ics.append(res.ic_tok[:n_inpaint])
                tok_times.append(res.timepoints[:n_inpaint])
            else:
                # NOTE: if there is end padding, it's dropped in the tok_ics
                ic_toks = res.ic_tok[n_inpaint-hop_size:n_inpaint]
                real_n_inpaint = res.ic_tok.shape[0]
                toks.append(res.tok[decoding_start+n_inpaint-hop_size:decoding_start+real_n_inpaint])
                tok_ics.append(ic_toks)
                tok_times.append(res.timepoints[n_inpaint-hop_size:n_inpaint])
    piece['toks'] = torch.cat(toks, axis=0)
    piece['tok_ics'] = torch.cat(tok_ics, axis=0)
    piece['tok_times'] = torch.cat(tok_times, axis=0)

In [89]:
piece_dict['kv331_1']['toks'].shape, piece_dict['kv331_1']['tok_times'].shape, piece_dict['kv331_1']['tok_ics'].shape

(torch.Size([6213, 4]), torch.Size([6213, 4]), torch.Size([6213, 4]))

In [81]:
(piece_dict['kv331_1']['toks'][:,0] == torch.tensor([dg.dataset.value2index['pitch'][i] for i in pitches])).all()

tensor(True)

In [158]:
list(Path(batik_dir, 'scores').glob('*'))

[PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv281_2.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv332_1.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv331_3.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv333_2.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv283_3.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv280_1.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv284_3.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv457_1.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv533_2.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv330_1.musicxml'),
 PosixPath('/share/hel/home/mathias/datasets/batik_plays_mozart/scores/kv333_3.musicxml'),

In [ ]:
snote_array = score.note_array()
# note array of the performance
pnote_array = performance.note_array()
# indices of the notes that have been matched
matched_note_idxs = pt.musicanalysis.performance_codec.get_matched_notes(
    spart_note_array=snote_array, 
    ppart_note_array=pnote_array, 
    alignment=alignment,
)

In [166]:
alignment[0]['score_id'][:-2]

'n1'

[{'label': 'match', 'score_id': 'n1-1', 'performance_id': 'n2'},
 {'label': 'match', 'score_id': 'n2-1', 'performance_id': 'n1'},
 {'label': 'match', 'score_id': 'n11-1', 'performance_id': 'n3'},
 {'label': 'match', 'score_id': 'n3-1', 'performance_id': 'n5'},
 {'label': 'match', 'score_id': 'n4-1', 'performance_id': 'n4'},
 {'label': 'match', 'score_id': 'n5-1', 'performance_id': 'n7'},
 {'label': 'match', 'score_id': 'n6-1', 'performance_id': 'n6'},
 {'label': 'match', 'score_id': 'n7-1', 'performance_id': 'n9'},
 {'label': 'match', 'score_id': 'n8-1', 'performance_id': 'n8'},
 {'label': 'match', 'score_id': 'n9-1', 'performance_id': 'n11'},
 {'label': 'match', 'score_id': 'n10-1', 'performance_id': 'n10'},
 {'label': 'match', 'score_id': 'n13-1', 'performance_id': 'n13'},
 {'label': 'match', 'score_id': 'n14-1', 'performance_id': 'n12'},
 {'label': 'match', 'score_id': 'n19-1', 'performance_id': 'n14'},
 {'label': 'match', 'score_id': 'n15-1', 'performance_id': 'n17'},
 {'label': 'm

In [1]:
onsets

NameError: name 'onsets' is not defined

In [178]:
from ic.curves import Interpolator
from jsonargparse import ActionConfigFile, ArgumentParser

from ic.curves.weight import Hann, Weight

# parser = ArgumentParser(parser_mode="jsonnet")
# parser.add_argument('--weight', type=dictWeight)
# parser.add_argument('--config', action=ActionConfigFile)
# jsonnet = '''
# {local channel_weight_mod = import './configs/weights.jsonnet',
# channel_weight_mod.hann(2.0, 'normalized_weights', ['pitch', 'time_shift'])}
# }
# '''
# args = parser.parse_args(['--config', jsonnet])
import numpy as np
# PITCHES = {
# 	0: ["C", "B#", "D--"],
# 	1: ["C#", "B##", "D-"],
# 	2: ["D", "C##", "E--"],
# 	3: ["D#", "E-", "F--"],
# 	4: ["E", "D##", "F-"],
# 	5: ["F", "E#", "G--"],
# 	6: ["F#", "E##", "G-"],
# 	7: ["G", "F##", "A--"],
# 	8: ["G#", "A-"],
# 	9: ["A", "G##", "B--"],
# 	10: ["A#", "B-", "C--"],
# 	11: ["B", "A##", "C-"],
# }

# accepted_pitches = [ii for i in PITCHES.values() for ii in i]
# pitch_to_ix = {p: accepted_pitches.index(p) for p in accepted_pitches}

# def tokenize_pitch_spelling(ps_note):
# 	# step = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6}[]
# 	alter = {0:"", 1:"#", 2:"##", -1:"-", -2:"--"}[ps_note["alter"].item()]
# 	return pitch_to_ix[ps_note["step"].item()+alter]

# def extract_features(perf_note):
# 	features = np.zeros((14,))
# 	# One hot of Pitch Class for first 12 entries
# 	features[int(perf_note["pitch"].item()%12)] = 1
# 	# pitch as float
# 	features[12] = perf_note["pitch"].item()/127
# 	# duration normalized per minute
# 	features[13] = perf_note["duration_sec"].item() / 60
# 	return features

for piece_name in piece_dict.keys():
    piece = piece_dict[piece_name]
    score = pt.load_score(Path(batik_dir, 'scores', f'{piece_name}.musicxml'))
    performance, alignment = pt.load_match(Path(batik_dir, 'match', f'{piece_name}.match'), first_note_at_zero=True)
    for dic in alignment:
        if dic['label'] == 'match':
            assert dic['score_id'][-2] == '-'
            dic['score_id'] = dic['score_id'][:-2]
    
    # score_file = Path(batik_dir, 'scores', f'{piece_name}.musicxml')
    # match_file = Path(batik_dir, 'match', f'{piece_name}.match')
    # performance, alignment = pt.load_match(match_file)
    # score = pt.load_score(score_file)
    # spart = pt.score.merge_parts(score)
    # spart = pt.score.unfold_part_maximal(spart, ignore_leaps=False)
    # matched_notes = [alignment[idx] for idx, d in enumerate(alignment) if d["label"] == "match"]
    # pna = performance.note_array()
    # sna = spart.note_array(include_pitch_spelling=True)
    # X, y = np.zeros((len(matched_notes), 14), dtype=float), np.zeros((len(matched_notes), ), dtype=int)
    # for idx, match_note in enumerate(matched_notes):
    #     X[idx] = extract_features(pna[np.where(pna["id"] == str(match_note["performance_id"]))])
    #     y[idx] = tokenize_pitch_spelling(sna[np.where(sna["id"] == match_note["score_id"])][["step", "alter", "octave"]])
    # perf_note_array = performance.note_array()
    kwargs = dict(include_pitch_spelling=True,include_key_signature=True)
    snote_array = score.note_array(**kwargs)
    # note array of the performance
    pnote_array = pt.utils.music.ensure_notearray(performance.note_array(), **kwargs)
    # indices of the notes that have been matched
    index_in_score_note_array, index_in_performance_notearray  = pt.musicanalysis.performance_codec.get_matched_notes(
        spart_note_array=snote_array, 
        ppart_note_array=pnote_array, 
        alignment=alignment,
    ).T
    names = ['step', 'alter', 'octave', 'ks_fifths', 'ks_mode']
    matched_array = np.lib.recfunctions.append_fields(pnote_array[index_in_performance_notearray], names=names, data=[snote_array[index_in_score_note_array][name] for name in names])
    assert pnote_array.shape[0] == piece['toks'].shape[0]
    # onsets = pnote_array['onset_sec']
    piece['interpolator'] = Interpolator(
        # NOTE: 
        metric_times = [piece['tok_times']],
        metric = [piece['tok_ics']],
        weight_fn = Hann(
            window_size=2.0,
            channel_weight=[0.9099181, 0, 0, 0.68306011]
        )
    )
    onsets = matched_array['onset_sec']
    tt = musan.estimate_tonaltension(matched_array, ss=onsets)
    # onsets = tt['onset_sec']
    ics = piece['interpolator']([torch.as_tensor(onsets)])[0]
    ics_summed = ics.sum(axis=1)
    feat = tt['cloud_diameter']
    corr = [pearsonr(tt[feat], ics_summed.numpy()) for feat in ['cloud_diameter', 'cloud_momentum', 'tensile_strain']]
    break
# interpolator = Interpolator(
#     metric_times = [piece_dict['kv331_1']['tok_times']],
#     metric = [piece_dict['kv331_1']['tok_ics']],
#     weight_fn = Hann(
#         window_size=2.0,
#         channel_weight=[0.9099181, 0, 0, 0.68306011]
#     )
# )

/share/hel/home/mathias/miniconda3/envs/pia2/lib/python3.8/site-packages/partitura/io/importmusicxml.py:585: UserWarning: ignoring empty <harmony> tag
  _handle_harmony(e, position, part)
/share/hel/home/mathias/miniconda3/envs/pia2/lib/python3.8/site-packages/partitura/io/importmusicxml.py:421: UserWarning: Found repeat without start
Starting point 0 is assumed
  warnings.warn(
/share/hel/home/mathias/devel/python3/PIA.No3/ic/curves/_curves.py:106: UserWarning: Most likely a cap needs to be set. For instance by calculating quantile .95
  warn('Most likely a cap needs to be set. For instance by calculating quantile .95')
Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 577, in _pydevd_bundle.pydevd_cython.PyDBFrame._handle_exception
  File "_pydevd_bundle/pydevd_cython.pyx", line 312, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/share/hel/home/mathias/miniconda3/envs/pia2/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd

ValueError: x and y must have the same length.

In [148]:
pearsonr

<function scipy.stats.stats.pearsonr(x, y)>

In [ ]:
tt

In [134]:
tt.dtype

dtype([('onset_sec', '<f4'), ('cloud_diameter', '<f4'), ('cloud_momentum', '<f4'), ('tensile_strain', '<f4')])

In [144]:
tt[10, 'onset_sec']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [136]:
tt[:10, ['onset_sec', 'cloud_diameter']]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

torch.Size([1997, 4])

In [123]:
len(performance.note_array())

1997

In [115]:
interpolator([torch.arange(0,10, 0.1)])[0]

tensor([[ 3.3390,  0.0000,  0.0000,  0.0000],
        [ 6.7259,  0.0000,  0.0000,  2.0885],
        [ 6.6187,  0.0000,  0.0000,  2.0603],
        [ 6.4316,  0.0000,  0.0000,  2.0071],
        [ 6.1694,  0.0000,  0.0000,  1.9302],
        [ 5.8384,  0.0000,  0.0000,  1.8316],
        [ 5.4467,  0.0000,  0.0000,  1.7137],
        [ 5.0041,  0.0000,  0.0000,  1.5793],
        [ 7.1186,  0.0000,  0.0000,  6.7323],
        [ 6.5829,  0.0000,  0.0000,  6.5185],
        [ 6.0002,  0.0000,  0.0000,  6.2352],
        [ 7.3348,  0.0000,  0.0000,  9.3339],
        [ 6.6730,  0.0000,  0.0000,  8.8817],
        [ 5.9862,  0.0000,  0.0000,  8.3445],
        [ 5.2912,  0.0000,  0.0000,  7.7355],
        [ 4.6052,  0.0000,  0.0000,  7.0696],
        [11.0506,  0.0000,  0.0000, 10.6391],
        [10.3600,  0.0000,  0.0000,  9.8720],
        [ 9.6412,  0.0000,  0.0000,  9.0481],
        [ 8.9117,  0.0000,  0.0000,  8.1877],
        [ 8.1896,  0.0000,  0.0000,  7.3121],
        [ 7.4645,  0.0000,  0.0000